# Library

In [357]:
from beb_chargers.scripts.script_helpers import build_trips_df, \
    build_charger_location_inputs
from beb_chargers.opt.charger_location import ChargerLocationModel
from beb_chargers.gtfs_beb import GTFSData
from beb_chargers.vis import plot_trips_and_terminals, plot_deadhead
from pathlib import Path
import datetime
import pandas as pd
import logging
logging.basicConfig(level=logging.INFO)

# Parameters

In [358]:
# Power output of each charger. Note that the model expects a unit of
# kilowatt-hours per minute, so that the amount of energy gained equals
# charging time (in minutes) multiplied by power (in kWh/min). Here, we
# use 450 kW = 450 / 60 kWh/min
chg_pwrs = 450 / 60

# Maximum number of chargers per site. We'll set it to 4 everywhere
n_max = 4

# Cost parameters. See TRC paper for where the values come from.
s_cost = 200000
c_cost = 698447
alpha = 190 * 365 * 12 / 60

# Locate CSV file giving candidate charger sites for this instance
site_fname = Path.cwd().parent / 'beb_chargers' / 'data' / 'test.csv'

# Load candidate charging sites given by Metro and add params as columns
loc_df = pd.read_csv(site_fname)
loc_df['max_chargers'] = n_max
loc_df['kw'] = chg_pwrs * 60
loc_df['fixed_cost'] = s_cost
loc_df['charger_cost'] = c_cost

# Define coordinates of overnight depot at South Base
depot_coords = (40.819197, -73.957060)

# Data

In [359]:
# Directory to GTFS files, as a platform-agnostic path
gtfs_dir = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS'

# Load GTFS data into our custom object
gtfs = GTFSData.from_dir(gtfs_dir)

In [360]:
# Battery capacity in kWh
battery_cap = 300
# Energy consumption per mile for all buses (we'll assume it's the same
# for all buses here)
kwh_per_mi = 3

In [361]:
import csv
import random

In [362]:
import csv
import random

def read_routes_file(filename):
    """
    Reads the routes file and returns a list of route IDs that start with 'M'.
    
    Args:
        filename (str): Path to the CSV file containing route IDs.
    
    Returns:
        list: Filtered list of route IDs where the first letter is 'M'.
    """
    route_ids = []
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if row['route_id'].startswith('M'):  # Filter routes that start with 'M'
                route_ids.append(row['route_id'])
    return route_ids

def random_select_routes(route_ids, percentage=1):
    """
    Randomly selects a subset of routes from the filtered list.
    
    Args:
        route_ids (list): List of filtered route IDs.
        percentage (float): Percentage of routes to select per iteration.
    
    Returns:
        list: Randomly selected route IDs.
    """
    num_to_select = round(len(route_ids) * percentage)
    return random.sample(route_ids, num_to_select) if route_ids else []

def main():
    filename = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS' / 'routes.txt'
    num_iterations = 10  # Define the number of iterations
    
    route_ids = read_routes_file(filename)  # Read and filter routes
    print(f"Total number of routes starting with 'M': {len(route_ids)}")
    
    all_results = []
    
    for i in range(num_iterations):
        selected = random_select_routes(route_ids)  # Select 20% of the filtered routes
        all_results.append(selected)
        print(f"\nIteration {i+1}:")
        print(f"Selected {len(selected)} routes: {selected}")
    
    return all_results

if __name__ == "__main__":
    random.seed(42)  # Ensure reproducibility
    
    # Run the main process
    results = main()
    
    print("\nFinal 2D array of selected route_ids:")
    for i, row in enumerate(results):
        print(f"Row {i+1}: {row}")

Total number of routes starting with 'M': 48

Iteration 1:
Selected 48 routes: ['M72', 'M7', 'MQAS', 'M11', 'M104', 'M103', 'M8', 'M5', 'M42', 'M4', 'M57', 'M21', 'M1', 'M99', 'M60+', 'M102', 'M86+', 'M106', 'M12', 'MVAS', 'M98', 'M191', 'M14A+', 'M125', 'M15+', 'M55', 'M34A+', 'M90', 'M34+', 'M116', 'M79+', 'M22', 'M35', 'M100', 'M20', 'M96', 'M3', 'M50', 'M2', 'M101', 'M66', 'M31', 'M9', 'M141A', 'M10', 'M15', 'M14D+', 'M23+']

Iteration 2:
Selected 48 routes: ['M90', 'M23+', 'M42', 'M7', 'M15', 'M4', 'M50', 'M116', 'M66', 'M14D+', 'M55', 'M101', 'M3', 'M1', 'M103', 'M72', 'M86+', 'M21', 'M98', 'M31', 'M2', 'M57', 'M8', 'M35', 'M125', 'M100', 'M34+', 'M14A+', 'M60+', 'M5', 'M15+', 'M99', 'M141A', 'M11', 'M12', 'M22', 'M106', 'M191', 'M34A+', 'M79+', 'M20', 'M104', 'M102', 'MVAS', 'MQAS', 'M96', 'M10', 'M9']

Iteration 3:
Selected 48 routes: ['M11', 'M3', 'M102', 'M55', 'M125', 'M99', 'M79+', 'M34+', 'M15+', 'M23+', 'M9', 'M106', 'M8', 'M104', 'M98', 'M21', 'M66', 'M100', 'M7', 'M90',

In [363]:
results = main()

Total number of routes starting with 'M': 48

Iteration 1:
Selected 48 routes: ['M90', 'M12', 'M34A+', 'M191', 'M86+', 'M20', 'M125', 'M15+', 'M116', 'M50', 'M8', 'M101', 'M96', 'M15', 'M100', 'M98', 'M72', 'M11', 'M60+', 'MVAS', 'M9', 'M21', 'M5', 'M102', 'M66', 'M99', 'M79+', 'M10', 'M103', 'M14A+', 'M14D+', 'M34+', 'M104', 'M55', 'M31', 'M35', 'M1', 'M141A', 'MQAS', 'M3', 'M4', 'M22', 'M23+', 'M42', 'M7', 'M106', 'M57', 'M2']

Iteration 2:
Selected 48 routes: ['M1', 'M104', 'M31', 'M66', 'M3', 'M23+', 'M20', 'M72', 'M55', 'M101', 'M15', 'M34+', 'M15+', 'M90', 'M9', 'MVAS', 'M5', 'M57', 'M102', 'M7', 'M4', 'M50', 'M14A+', 'M106', 'M103', 'MQAS', 'M11', 'M22', 'M2', 'M14D+', 'M96', 'M116', 'M10', 'M60+', 'M125', 'M35', 'M100', 'M8', 'M98', 'M42', 'M86+', 'M79+', 'M21', 'M12', 'M141A', 'M99', 'M34A+', 'M191']

Iteration 3:
Selected 48 routes: ['M35', 'M34+', 'M72', 'M104', 'M11', 'M22', 'M3', 'M116', 'M98', 'M96', 'M141A', 'M125', 'M42', 'M4', 'M8', 'M9', 'M103', 'M101', 'M14A+', 'M79+

In [364]:
# All routes included in analysis
beb_routes = [
    'M1', 'M2', 'M3', 'M4', 'M5', 'M7', 'M8', 'M9', 'M10',
    'M100', 'M101', 'M102', 'M103', 'M104',
    'M14A-SBS', 'M14D-SBS', 'M15-SBS', 'M60-SBS',
    'Q30', 'Bx18B', 'Bx41-SBS', 'Bx36', 'B44-SBS',
    'B52', 'B74', 'X37', 'B111'
]
beb_routes = [str(r) for r in beb_routes]

beb_routes = results[0]


import csv
import random

def read_routes_file(filename):
    """
    Read routes file and return route_ids list
    Args:
        filename: CSV path
    Returns:
        route_ids list
    """
    route_ids = []
    with open(filename, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            route_ids.append(row['route_id'])
    return route_ids

def filter_m_routes(route_ids):
    """
    Filter routes that start with 'M'
    Args:
        route_ids: list of all route IDs
    Returns:
        list of route IDs starting with 'M'
    """
    return [route for route in route_ids if route.startswith('M')]

def random_select_routes_with_m_first(route_ids, percentage=0.1):
    """
    Select routes with first one being M-route
    Args:
        route_ids: all routes ID
        percentage: percentage to select
    Returns:
        selected routes ID list starting with an M-route
    """
    # Separate M routes and non-M routes
    m_routes = filter_m_routes(route_ids)
    non_m_routes = [r for r in route_ids if not r.startswith('M')]
    
    if not m_routes:
        raise ValueError("No M-routes found in the input data")
    
    # Calculate how many additional routes to select after M-route
    total_to_select = round(len(route_ids) * percentage)
    remaining_to_select = total_to_select - 1  # -1 because we'll select one M-route
    
    # Select one M-route for the first position
    selected_routes = [random.choice(m_routes)]
    
    # Select remaining routes from all available routes (excluding the selected M-route)
    available_routes = [r for r in route_ids if r != selected_routes[0]]
    
    # Randomly select the remaining routes
    for _ in range(remaining_to_select):
        if not available_routes:
            break
        idx = random.randrange(len(available_routes))
        selected_routes.append(available_routes.pop(idx))
    
    return selected_routes

def main():
    filename = Path.cwd().parent / 'beb_chargers' / 'data' / 'gtfs' / 'GTFS' / 'routes.txt'
    num_iterations = 10
    
    route_ids = read_routes_file(filename)
    print(f"Total number of routes: {len(route_ids)}")
    print(f"Number of M-routes: {len(filter_m_routes(route_ids))}")
    
    all_results = []
    
    for i in range(num_iterations):
        selected = random_select_routes_with_m_first(route_ids)
        all_results.append(selected)
        print(f"\nIteration {i+1}:")
        print(f"Selected {len(selected)} routes: {selected}")
        print(f"First route starts with 'M': {selected[0].startswith('M')}")
    
    return all_results

if __name__ == "__main__":
    random.seed(42)
    
    # Run main program
    results = main()
    
    print("\nFinal 2D array of selected route_ids:")
    for i, row in enumerate(results):
        print(f"Row {i+1}: {row}")

In [365]:
# We'll run analysis for March 28, 2024
ocl_date = datetime.datetime(2025, 3, 14)
# Call helper function to build up DF
beb_trips = build_trips_df(
    gtfs=gtfs,
    date=ocl_date,
    routes=beb_routes,
    depot_coords=depot_coords,
    add_depot_dh=True,
    add_kwh_per_mi=False,
    add_durations=False,
    routes_60=[]
)
# Add a column giving energy consumption for each trip
beb_trips['kwh_per_mi'] = kwh_per_mi

In [366]:
beb_trips

,trip_id,route_id,service_id,block_id,shape_id,route,route_type,route_desc,start_time,end_time,trip_idx,start_lat,start_lon,end_lat,end_lon,total_dist,duration_sched,60_dummy,kwh_per_mi
0,OH_A5-Saturday-BM-141800_M15_201,M15,OH_A5-Saturday-BM,36440938,M150023,M15,3,via 1st Av / 2nd Av,2025-03-14 23:38:00,2025-03-15 00:39:00,0,40.803182,-73.932486,40.701549,-74.012199,20.429058,61.0,0,3
1,OH_A5-Saturday-BM-143500_M101_1,M103,OH_A5-Saturday-BM,36441011,M1030040,M103,3,via 3rd Av / Lexington Av,2025-03-14 23:55:00,2025-03-15 00:37:00,0,40.803858,-73.937786,40.711699,-74.007341,18.179389,42.0,0,3
2,MV_A5-Saturday-BM-139500_M104_101,M104,MV_A5-Saturday-BM,36441665,M1040167,M104,3,via Broadway / 8th Av,2025-03-14 23:15:00,2025-03-14 23:56:00,0,40.814222,-73.953091,40.756384,-73.989832,11.766449,41.0,0,3
3,OH_A5-Weekday-SDon-006000_M15_201,M15,OH_A5-Weekday-SDon,36441933,M150044,M15,3,via 1st Av / 2nd Av,2025-03-14 01:00:00,2025-03-14 01:43:00,0,40.701481,-74.012461,40.803102,-73.932299,18.574302,43.0,0,3
4,OH_A5-Weekday-SDon-012000_M15_201,M15,OH_A5-Weekday-SDon,36441933,M150023,M15,3,via 1st Av / 2nd Av,2025-03-14 02:00:00,2025-03-14 02:46:00,1,40.803182,-73.932486,40.701549,-74.012199,8.641575,46.0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6188,KB_A5-Weekday-SDon-082400_M100_14,M100,KB_A5-Weekday-SDon,36466228,M1000107,M100,3,via Broadway / Amsterdam Av,2025-03-14 13:44:00,2025-03-14 14:44:00,1,40.810959,-73.953041,40.872464,-73.912582,5.885478,60.0,0,3
6189,KB_A5-Weekday-SDon-089400_M100_14,M100,KB_A5-Weekday-SDon,36466228,M1000113,M100,3,via Broadway / Amsterdam Av,2025-03-14 14:54:00,2025-03-14 16:04:00,2,40.872558,-73.912473,40.810293,-73.952930,6.224719,70.0,0,3
6190,KB_A5-Weekday-SDon-098400_M100_14,M100,KB_A5-Weekday-SDon,36466228,M1000107,M100,3,via Broadway / Amsterdam Av,2025-03-14 16:24:00,2025-03-14 17:27:00,3,40.810959,-73.953041,40.872464,-73.912582,12.180382,63.0,0,3
6191,KB_A5-Weekday-SDon-048000_M100_20,M100,KB_A5-Weekday-SDon,36466229,M1000113,M100,3,via Broadway / Amsterdam Av,2025-03-14 08:00:00,2025-03-14 09:01:00,0,40.872558,-73.912473,40.810293,-73.952930,12.784091,61.0,0,3


In [367]:
# Record how many trips and blocks are active on our specified routes on this day
logging.info(
    '{}: There are {} total trips to be served by {} BEB blocks.'.format(
        ocl_date.strftime('%m/%d/%y'), len(beb_trips), beb_trips['block_id'].nunique()
    )
)

INFO:root:03/14/25: There are 6193 total trips to be served by 699 BEB blocks.


In [368]:
# Create a map of the problem instance
inst_map = plot_trips_and_terminals(
    beb_trips, loc_df, gtfs.shapes_df, 'light'
)
# These config params make map downloads higher resolution
config = {
    'toImageButtonOptions': {
        'format': 'png',
        'scale': 3
    }
}
inst_map.show(config=config)

In [369]:
opt_kwargs = build_charger_location_inputs(
    gtfs=gtfs,
    trips_df=beb_trips,
    chargers_df=loc_df,
    depot_coords=depot_coords,
    battery_cap=battery_cap
)

INFO:gtfs_data:OSM request: 48 origins, 37 destinations (1776 total routes)
INFO:gtfs_data:OpenRouteService matrix returned in 1.01 seconds.
INFO:gtfs_data:OSM request: 37 origins, 53 destinations (1961 total routes)
INFO:gtfs_data:OpenRouteService matrix returned in 0.99 seconds.


In [370]:
# this is the dictionary
type (opt_kwargs)

dict

In [371]:
# Create instance of charger location model based on those inputs
clm = ChargerLocationModel(**opt_kwargs)
# Solve the model with zero optimality gap
clm.solve(alpha=alpha, opt_gap=0, bu_kwh=battery_cap)

INFO:charger_location:Number of blocks that require charging: 71
INFO:charger_location:Number of trips in charging blocks: 1192
INFO:charger_location:Number of infeasible blocks: 4
INFO:charger_location:Infeasible block IDs: ['36443130', '36443131', '36443133', '36443135']
INFO:charger_location:Number of trips in infeasible blocks: 82
INFO:charger_location:Time for heuristic runs: 0.31 seconds
INFO:charger_location:Average number of backups: 2.06
INFO:charger_location:Minimum number of backups: 2

INFO:charger_location:Mean Conflict set size: 7.90
INFO:gurobipy:Read LP format model from file C:\Users\chris\AppData\Local\Temp\tmpuqd64n84.pyomo.lp
INFO:gurobipy:Reading time = 0.07 seconds
INFO:gurobipy:x1: 22219 rows, 19516 columns, 82211 nonzeros
INFO:gurobipy:Set parameter MIPGap to value 0
INFO:gurobipy:Set parameter MIPFocus to value 3
INFO:gurobipy:Set parameter Method to value 1
INFO:gurobipy:Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (26100.2))
INFO:gur

In [372]:
clm.log_results()

INFO:charger_location:Total number of backup buses used: 2
INFO:charger_location:Optimal objective function value: 10047273.93
INFO:charger_location:Optimal stations: ['207th Street Station', '3rd Avenue138th Street Station']
INFO:charger_location:Number of chargers: {'149th StreetGrand Concourse': 0, '161st StreetYankee Stadium Station': 0, '167th Street Station': 0, '174th175th Streets Station': 0, '182nd183rd Streets Station': 0, '207th Street Station': 1, '215th Street Station': 0, '3rd Avenue138th Street Station': 1, 'Atlantic Terminal': 0, 'Brooklyn College / Flatbush Avenue Terminal': 0, 'Bus Stops around LaGuardia Airport': 0, 'Co-op City Bay Plaza': 0, 'Coney Island  Stillwell Avenue Terminal': 0, 'Dyckman Street Station': 0, 'Eltingville Transit Center': 0, 'Far Rockaway  Mott Avenue': 0, 'Flushing  Main Street': 0, 'Fordham Plaza': 0, 'Gateway Center (Near Spring Creek)': 0, 'Jamaica Center  Parsons/Archer': 0, 'Kew Gardens  Union Tpke': 0, 'Near Kingsbridge Bus Depot': 0, '

In [373]:
clm.to_df()

,block_id,trip_id,trip_idx,start_time,end_time,soc,chg_site,chg_time,dh1,dh2,dh3
0,36441937,OH_A5-Weekday-SDon-023000_M15_205,1,65848550.0,65848600.0,293.272896,NaN,NaN,NaN,NaN,NaN
1,36441937,OH_A5-Weekday-SDon-029000_M15_205,2,65848610.0,65848657.0,260.578257,NaN,NaN,NaN,NaN,NaN
2,36441937,OH_A5-Weekday-SDon-035000_M15_205,3,65848670.0,65848721.0,234.714226,NaN,NaN,NaN,NaN,NaN
3,36441937,OH_A5-Weekday-SDon-040700_M15_205,4,65848727.0,65848791.0,212.262560,NaN,NaN,NaN,NaN,NaN
4,36441937,OH_A5-Weekday-SDon-049100_M15_205,5,65848811.0,65848896.0,190.297151,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1097,36466220,100,15,65849547.0,65849547.0,22.571407,NaN,NaN,NaN,NaN,NaN
1098,36466221,0,0,65848560.0,65848560.0,300.000000,NaN,NaN,NaN,NaN,NaN
1099,36466221,100,15,65849510.0,65849510.0,45.071407,NaN,NaN,NaN,NaN,NaN
1100,36466226,0,0,65848620.0,65848620.0,300.000000,NaN,NaN,NaN,NaN,NaN


In [374]:
# clm.plot_chargers()

In [375]:
fig = plot_deadhead(
    result_df=clm.to_df(), loc_df=loc_df, coords_df=beb_trips
)
fig.show()

INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
INFO:googlemaps.client:API queries_quota: 60
